In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.datasets import load_digits
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Calculating the margin

In [2]:
def compute_margin(X, y, w, b):
    margin = []
    for i in range(y.shape[0]):
        margin.append((y[i]*(np.dot(X[i,:], w)+b))/sqrt(sum(w**2))) 
    if min(margin) >= 0:
        return min(margin)
    return 0

## ELM-IMA with W_init = [0, ..., 0] 

In [3]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin
class ELM_IMA(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        norm_w = sqrt(sum(w**2))
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * norm_w - self.lambda_param * alpha[i]):
                    if norm_w != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/norm_w
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w * lambda_t + self.eta * y[i] * X[i,:]
                    norm_w = sqrt(sum(w**2))
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.w_elm.shape[0])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = sqrt(sum(self.w**2))
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
        return t, updates, iterations, margin

    # Function that manage the training of IMA ELM
    def fit(self, X, y):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        self.Z = np.array([random.uniform(-1, 1) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        self.H = np.tanh(np.dot(X, self.Z))
        w = np.dot(np.linalg.pinv(self.H), y)  
        self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations, margin = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations, margin
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        else:
            y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## ELM-IMA with W_init = $\beta$

In [4]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin
class ELM_IMA_b(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        norm_w = sqrt(sum(w**2))
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * norm_w - self.lambda_param * alpha[i]):
                    if norm_w != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/norm_w
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w * lambda_t + self.eta * y[i] * X[i,:]
                    norm_w = sqrt(sum(w**2))
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = deepcopy(self.w_elm)
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)            
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = sqrt(sum(self.w**2))
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
        return t, updates, iterations, margin

    # Function that manage the training of IMA ELM   
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-1, 1) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        w = np.dot(np.linalg.pinv(self.H), y)  
        self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations, margin = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations, margin
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        else:
            y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## Commom ELM

In [5]:
class ELM(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
    
    def fit(self, X, y):
        # Adding polarization term 
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        n = X_new.shape[1]
        self.Z = np.array([random.uniform(-1, 1) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        H = np.tanh(np.dot(X_new, self.Z))
        self.w = np.dot(np.linalg.pinv(H), y)           
        return self.w, H, self.Z
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        y_predicted = np.sign(np.dot(H, self.w))
        y_predicted[y_predicted==0]=1
        return y_predicted

## Function to get Results

In [6]:
def grid_C(X, y):
    # GridSearch for C parameter of SVM:
    parameters = {'C':np.logspace(-3, 2, 50)}
    clf = svm.SVC(kernel="rbf", gamma='scale')
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=10, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['C']

In [41]:
def grid_l(X, y, p):
    # GridSearch for lambda and learning rate of IMA ELM
    parameters = {'lambda_param':np.linspace(0.01, 100, 50)}
    clf = ELM_IMA(n_neurons=p, delta_margin=10^-3, IMA_iterations=20, max_updates=10000)
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=10, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['lambda_param']
    

In [ ]:
def results(X, y, n_splits, p, eta, IMA_iterations, lambda_param, C):      
    print(f'Parameters: p={p}, eta={eta}, lambda={lambda_param}, C={C}')
    # Stratified k fold cross validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
    i=0
    
    train_accuracy_IM_ELM = np.zeros(n_splits)
    test_accuracy_IM_ELM = np.zeros(n_splits)
    margin_IM_ELM = np.zeros(n_splits)
    updates = np.zeros(n_splits)
    iterations_FMP = np.zeros(n_splits) 
    iterations_IMA = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_b = np.zeros(n_splits)
    test_accuracy_IM_ELM_b = np.zeros(n_splits)
    margin_IM_ELM_b = np.zeros(n_splits)
    updates_b = np.zeros(n_splits)
    iterations_FMP_b = np.zeros(n_splits) 
    iterations_IMA_b = np.zeros(n_splits)
          
    train_accuracy_ELM = np.zeros(n_splits)
    test_accuracy_ELM = np.zeros(n_splits)
    margin_ELM = np.zeros(n_splits)
    
    train_accuracy_SVM = np.zeros(n_splits)
    test_accuracy_SVM = np.zeros(n_splits)
    margin_SVM = np.zeros(n_splits)

    margins=[]
    margins_b = []
        
    for train_index, test_index in kf.split(X, y):
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]
               
        # ELM-IMA
        clf = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3, IMA_iterations=IMA_iterations, max_updates=10000)
        iterations_IMA[i], updates[i], iterations_FMP[i], margin = clf.fit(X_train, y_train)
        margins.append(margin)
        y_hat=clf.predict(X_test, use_IMA_w = True)
        y_hat_train=clf.predict(X_train, use_IMA_w = True)
        margin_IM_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, clf.b)
        train_accuracy_IM_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM[i] = accuracy_score(y_test, y_hat)
        Z = clf.Z
                
        # ELM
        y_hat=clf.predict(X_test, use_IMA_w = False)
        y_hat_train=clf.predict(X_train, use_IMA_w = False)
        margin_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w_elm, 0)
        train_accuracy_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_ELM[i] = accuracy_score(y_test, y_hat)
                
         # ELM-IMA (beta)
        clf = ELM_IMA_b(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3, IMA_iterations=IMA_iterations, max_updates=10000)
        iterations_IMA_b[i], updates_b[i], iterations_FMP_b[i], margin_b = clf.fit(X_train, y_train, Z)
        margins_b.append(margin_b)
        y_hat=clf.predict(X_test, use_IMA_w = True)
        y_hat_train=clf.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_b[i] = compute_margin(clf.H[:,:], y_train, clf.w, clf.b)
        train_accuracy_IM_ELM_b[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_b[i] = accuracy_score(y_test, y_hat)
                     
        # SVC Classifier
        clf = svm.SVC(kernel="rbf", C=C, gamma='scale')
        clf.fit(X_train, y_train)
        y_hat=clf.predict(X_test)
        y_hat_train=clf.predict(X_train)
        train_accuracy_SVM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_SVM[i] = accuracy_score(y_test, y_hat)
        i+=1
        
    print("********* Results ELM-IMA **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA.std()))
    print("Updates: " + '{:.4f}'.format(updates.mean())+ "+/-" + '{:.4f}'.format(updates.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM.std()))
    
    print("********* Results ELM-IMA Beta **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_b.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_b.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_b.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_b.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_b.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_b.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_b.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_b.std()))
    print("Updates: " + '{:.4f}'.format(updates_b.mean())+ "+/-" + '{:.4f}'.format(updates_b.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_b.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_b.std()))
       
    print("********* Results ELM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_ELM.std()))
    print("Margin: " + '{:.15f}'.format(margin_ELM.mean())+ "+/-" + '{:.15f}'.format(margin_ELM.std()))
    
    print("********* Results SVM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_SVM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_SVM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_SVM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_SVM.std()))
    print("Margin: " + '{:.6f}'.format(margin_SVM.mean())+ "+/-" + '{:.6f}'.format(margin_SVM.std()))
    return margins

In [42]:
def run(X,y):
    # Normalizing data:
    normalizer = MinMaxScaler()
    X = normalizer.fit_transform(X)
    C = grid_C(X, y)
    n = len(X)
    if n>1000:
        n=1000
    for p in [n, int(n/2), int(n/3)]:
        l = grid_l(X, y, p=p)
        print(f"Experimento com {p} neurônios:" )
        m = results(X, y, n_splits=10, p=p, eta=0.1, IMA_iterations=20, lambda_param=l, C=C)
        print("\n\n")

In [10]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1
run(X, y)
#plot_margin_evolution(m)

Experimento com 150 neurônios:
Parameters: p=150, eta=0.1, lambda=0.01, C=0.0339322177189533
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 3590.9000+/-2284.9614
Iterations IMA: 19.4000+/-1.8000
Updates: 9305.0000+/-5648.4802
Margin: 1.118911450+/-0.074857915
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 72.0000+/-14.6082
Iterations IMA: 20.0000+/-0.0000
Updates: 121.4000+/-20.7374
Margin: 0.000323332+/-0.000098410
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9933+/-0.0200
Margin: 0.000019436+/-0.000004903
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 75 neurônios:
Parameters: p=75, eta=0.1, lambda=0.01, C=0.0339322177189533
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 1762.5000+/-795

In [11]:
synthetic_dataset = pd.read_csv('data/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))
run(X, y)

Experimento com 600 neurônios:
Parameters: p=600, eta=0.1, lambda=2.050612244897959, C=0.02682695795279726
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9950+/-0.0150
Iterations: 176.6000+/-24.3442
Iterations IMA: 20.0000+/-0.0000
Updates: 1772.4000+/-366.1380
Margin: 0.515368636+/-0.063980023
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9933+/-0.0153
Iterations: 120.1000+/-11.8697
Iterations IMA: 20.0000+/-0.0000
Updates: 910.8000+/-167.7700
Margin: 0.325584045+/-0.081658744
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6750+/-0.0672
Margin: 0.012283029+/-0.001930045
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 300 neurônios:
Parameters: p=300, eta=0.1, lambda=36.74102040816326, C=0.02682695795279726
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9883+/-0.0183

In [43]:
robot_dataset = pd.read_csv('data/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))
run(X, y)

Experimento com 117 neurônios:
Parameters: p=117, eta=0.1, lambda=0.01, C=100.0
********* Results ELM-IMA **************
Acc train: 0.8463+/-0.0836
Acc test: 0.8038+/-0.0358
Iterations: 637.3000+/-537.3206
Iterations IMA: 4.9000+/-7.5558
Updates: 12808.1000+/-5305.4077
Margin: 0.000610744+/-0.001230155
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6159+/-0.1445
Iterations: 444.8000+/-186.3437
Iterations IMA: 6.9000+/-8.5843
Updates: 10066.4000+/-1811.7923
Margin: 0.000001843+/-0.000000698
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6159+/-0.1445
Margin: 0.000001923+/-0.000000703
********* Results SVM **************
Acc train: 0.9439+/-0.0271
Acc test: 0.8712+/-0.1065
Margin: 0.000000+/-0.000000



Experimento com 58 neurônios:
Parameters: p=58, eta=0.1, lambda=2.050612244897959, C=100.0
********* Results ELM-IMA **************
Acc train: 0.8111+/-0.0330
Acc test: 0.8038+/-0.0517
Iterations: 411.8000+/-119.0620
Itera

In [13]:
df = pd.read_csv('data/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=4.0875510204081635, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 62.9000+/-7.7389
Iterations IMA: 20.0000+/-0.0000
Updates: 1357.7000+/-397.6848
Margin: 0.049897723+/-0.022380742
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 60.0000+/-4.7117
Iterations IMA: 20.0000+/-0.0000
Updates: 1504.2000+/-473.8898
Margin: 0.065234755+/-0.007324467
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.058185186+/-0.002137417
********* Results SVM **************
Acc train: 0.9990+/-0.0005
Acc test: 0.9972+/-0.0016
Margin: 0.000000+/-0.000000



Experimento com 500 neurônios:
Parameters: p=500, eta=0.1, lambda=4.291428571428571, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0001
Acc test: 0.9998+/-0.0005

In [14]:
ionosphere_dataset = pd.read_csv('data/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 351 neurônios:
Parameters: p=351, eta=0.1, lambda=8.172448979591836, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.9918+/-0.0091
Acc test: 0.9288+/-0.0545
Iterations: 311.8000+/-33.2379
Iterations IMA: 20.0000+/-0.0000
Updates: 2937.4000+/-344.5534
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9956+/-0.0015
Acc test: 0.8775+/-0.0712
Iterations: 248.2000+/-15.3805
Iterations IMA: 20.0000+/-0.0000
Updates: 2240.6000+/-265.2268
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6353+/-0.0874
Margin: 0.002620166+/-0.000790723
********* Results SVM **************
Acc train: 0.9769+/-0.0038
Acc test: 0.9544+/-0.0291
Margin: 0.000000+/-0.000000



Experimento com 175 neurônios:
Parameters: p=175, eta=0.1, lambda=65.3095918367347, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.9759+/-0.0108
Acc test: 0.9145+/-0.0313
I

In [15]:
# read in banknote authentication set
banknotes = pd.read_csv('data/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=0.01, C=0.7196856730011522
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 2239.2000+/-1185.5485
Iterations IMA: 13.3000+/-3.9000
Updates: 33732.9000+/-14851.0482
Margin: 0.141611144+/-0.008639563
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9956+/-0.0036
Iterations: 505.2000+/-103.4290
Iterations IMA: 20.0000+/-0.0000
Updates: 6828.9000+/-1203.4206
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9380+/-0.0276
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.9996+/-0.0012
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 500 neurônios:
Parameters: p=500, eta=0.1, lambda=0.01, C=0.7196856730011522
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 2432

In [16]:
wine_dataset = pd.read_csv('data/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
run(X, y)

Experimento com 178 neurônios:
Parameters: p=178, eta=0.1, lambda=22.456734693877554, C=0.9102981779915218
********* Results ELM-IMA **************
Acc train: 0.9994+/-0.0019
Acc test: 0.9722+/-0.0373
Iterations: 82.2000+/-8.9867
Iterations IMA: 20.0000+/-0.0000
Updates: 334.3000+/-56.7945
Margin: 0.071665807+/-0.062555477
********* Results ELM-IMA Beta **************
Acc train: 0.9994+/-0.0019
Acc test: 0.9722+/-0.0278
Iterations: 72.9000+/-5.8387
Iterations IMA: 20.0000+/-0.0000
Updates: 251.3000+/-34.6729
Margin: 0.016616692+/-0.008253451
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7467+/-0.1194
Margin: 0.007633024+/-0.001569527
********* Results SVM **************
Acc train: 0.9950+/-0.0025
Acc test: 0.9833+/-0.0356
Margin: 0.000000+/-0.000000



Experimento com 89 neurônios:
Parameters: p=89, eta=0.1, lambda=97.95938775510204, C=0.9102981779915218
********* Results ELM-IMA **************
Acc train: 0.9944+/-0.0052
Acc test: 0.9778+/-0.0369
Iteration

In [17]:
wdbc_dataset = pd.read_csv('data/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 569 neurônios:
Parameters: p=569, eta=0.1, lambda=65.3095918367347, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.9811+/-0.0071
Acc test: 0.9736+/-0.0197
Iterations: 140.8000+/-14.3722
Iterations IMA: 20.0000+/-0.0000
Updates: 1488.5000+/-178.8325
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9869+/-0.0030
Acc test: 0.9718+/-0.0162
Iterations: 146.4000+/-25.0408
Iterations IMA: 20.0000+/-0.0000
Updates: 1338.2000+/-202.2512
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7240+/-0.0599
Margin: 0.002462136+/-0.000324350
********* Results SVM **************
Acc train: 0.9879+/-0.0015
Acc test: 0.9842+/-0.0214
Margin: 0.000000+/-0.000000



Experimento com 284 neurônios:
Parameters: p=284, eta=0.1, lambda=100.0, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.9852+/-0.0043
Acc test: 0.9719+/-0.0179
Iterations: 9

In [18]:
sonar_dataset = pd.read_csv('data/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 208 neurônios:
Parameters: p=208, eta=0.1, lambda=4.091224489795918, C=7.543120063354623
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6929+/-0.1436
Iterations: 207.8000+/-25.4708
Iterations IMA: 20.0000+/-0.0000
Updates: 2378.2000+/-262.0877
Margin: 0.081877328+/-0.027749915
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7217+/-0.1489
Iterations: 176.4000+/-32.5245
Iterations IMA: 20.0000+/-0.0000
Updates: 1985.9000+/-364.4424
Margin: 0.059882911+/-0.027394289
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6171+/-0.1737
Margin: 0.027286876+/-0.008034951
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6926+/-0.1347
Margin: 0.000000+/-0.000000



Experimento com 104 neurônios:
Parameters: p=104, eta=0.1, lambda=10.213061224489795, C=7.543120063354623
********* Results ELM-IMA **************
Acc train: 0.9728+/-0.0150
Acc test: 0.7031+/-0.1389
I

In [19]:
pima_dataset = pd.read_csv('data/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 768 neurônios:
Parameters: p=768, eta=0.1, lambda=63.26897959183673, C=0.22229964825261955
********* Results ELM-IMA **************
Acc train: 0.7883+/-0.0127
Acc test: 0.7761+/-0.0237
Iterations: 305.5000+/-29.9341
Iterations IMA: 20.0000+/-0.0000
Updates: 18180.4000+/-922.4899
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.8035+/-0.0236
Acc test: 0.6900+/-0.0349
Iterations: 276.1000+/-20.0522
Iterations IMA: 20.0000+/-0.0000
Updates: 15992.4000+/-737.1783
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5505+/-0.0495
Margin: 0.000006594+/-0.000000400
********* Results SVM **************
Acc train: 0.7863+/-0.0061
Acc test: 0.7747+/-0.0374
Margin: 0.000000+/-0.000000



Experimento com 384 neurônios:
Parameters: p=384, eta=0.1, lambda=67.35020408163265, C=0.22229964825261955
********* Results ELM-IMA **************
Acc train: 0.7785+/-0.0163
Acc test: 0.7487+/-0.0

In [20]:
statlog_dataset = pd.read_csv('data/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 270 neurônios:
Parameters: p=270, eta=0.1, lambda=75.5126530612245, C=0.08685113737513529
********* Results ELM-IMA **************
Acc train: 0.8951+/-0.0149
Acc test: 0.7926+/-0.0579
Iterations: 154.9000+/-10.0145
Iterations IMA: 20.0000+/-0.0000
Updates: 2369.0000+/-140.9525
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9202+/-0.0115
Acc test: 0.8037+/-0.0643
Iterations: 144.9000+/-13.3675
Iterations IMA: 20.0000+/-0.0000
Updates: 2005.1000+/-91.9951
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5778+/-0.0940
Margin: 0.007204195+/-0.001940335
********* Results SVM **************
Acc train: 0.8370+/-0.0076
Acc test: 0.8333+/-0.0504
Margin: 0.000000+/-0.000000



Experimento com 135 neurônios:
Parameters: p=135, eta=0.1, lambda=83.67510204081633, C=0.08685113737513529
********* Results ELM-IMA **************
Acc train: 0.8794+/-0.0125
Acc test: 0.8111+/-0.0584


In [21]:
mammo = pd.read_csv('data/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 830 neurônios:
Parameters: p=830, eta=0.1, lambda=18.375510204081632, C=100.0
********* Results ELM-IMA **************
Acc train: 0.8305+/-0.0053
Acc test: 0.8096+/-0.0562
Iterations: 490.1000+/-15.1291
Iterations IMA: 20.0000+/-0.0000
Updates: 46275.2000+/-1710.0074
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.8415+/-0.0044
Acc test: 0.7458+/-0.0483
Iterations: 478.7000+/-22.9349
Iterations IMA: 20.0000+/-0.0000
Updates: 43165.2000+/-1934.6814
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.9151+/-0.0069
Acc test: 0.7217+/-0.0489
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.8398+/-0.0062
Acc test: 0.8145+/-0.0464
Margin: 0.000000+/-0.000000



Experimento com 415 neurônios:
Parameters: p=415, eta=0.1, lambda=6.131836734693877, C=100.0
********* Results ELM-IMA **************
Acc train: 0.8282+/-0.0099
Acc test: 0.8193+/-0.0536
Iterations: 604.8000+

In [22]:
haberman = pd.read_csv('data/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 306 neurônios:
Parameters: p=306, eta=0.1, lambda=2.050612244897959, C=1.151395399326448
********* Results ELM-IMA **************
Acc train: 0.7480+/-0.0091
Acc test: 0.7484+/-0.0287
Iterations: 170.3000+/-58.1482
Iterations IMA: 3.0000+/-1.6125
Updates: 14476.9000+/-4202.2262
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9089+/-0.0502
Acc test: 0.5457+/-0.0674
Iterations: 428.1000+/-78.5410
Iterations IMA: 20.0000+/-0.0000
Updates: 22117.3000+/-5079.2011
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.9779+/-0.0030
Acc test: 0.5261+/-0.0648
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.7669+/-0.0082
Acc test: 0.7551+/-0.0249
Margin: 0.000000+/-0.000000



Experimento com 153 neurônios:
Parameters: p=153, eta=0.1, lambda=69.39081632653061, C=1.151395399326448
********* Results ELM-IMA **************
Acc train: 0.7407+/-0.0097
Acc test: 0.7451+/-0.0320

In [23]:
transfusion = pd.read_csv('data/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 748 neurônios:
Parameters: p=748, eta=0.1, lambda=38.78163265306122, C=0.28117686979742307
********* Results ELM-IMA **************
Acc train: 0.7699+/-0.0049
Acc test: 0.7674+/-0.0253
Iterations: 142.8000+/-111.1025
Iterations IMA: 4.3000+/-4.9406
Updates: 13238.0000+/-4634.5127
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.7909+/-0.0206
Acc test: 0.6658+/-0.1076
Iterations: 506.8000+/-185.0934
Iterations IMA: 20.0000+/-0.0000
Updates: 16000.2000+/-1988.0963
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.9067+/-0.0045
Acc test: 0.5790+/-0.1035
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.7653+/-0.0016
Acc test: 0.7647+/-0.0084
Margin: 0.000000+/-0.000000



Experimento com 374 neurônios:
Parameters: p=374, eta=0.1, lambda=46.94408163265306, C=0.28117686979742307
********* Results ELM-IMA **************
Acc train: 0.7681+/-0.0030
Acc test: 0.7727+/-

In [24]:
australian = pd.read_csv('data/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 690 neurônios:
Parameters: p=690, eta=0.1, lambda=34.7004081632653, C=7.543120063354623
********* Results ELM-IMA **************
Acc train: 0.9148+/-0.0038
Acc test: 0.8449+/-0.0384
Iterations: 376.6000+/-20.0609
Iterations IMA: 20.0000+/-0.0000
Updates: 15257.6000+/-482.4975
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9262+/-0.0055
Acc test: 0.6348+/-0.0593
Iterations: 344.2000+/-20.6775
Iterations IMA: 20.0000+/-0.0000
Updates: 13506.8000+/-686.5924
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5261+/-0.0554
Margin: 0.000056068+/-0.000014701
********* Results SVM **************
Acc train: 0.9084+/-0.0046
Acc test: 0.8638+/-0.0350
Margin: 0.000000+/-0.000000



Experimento com 345 neurônios:
Parameters: p=345, eta=0.1, lambda=61.228367346938775, C=7.543120063354623
********* Results ELM-IMA **************
Acc train: 0.8987+/-0.0085
Acc test: 0.8594+/-0.0304


In [25]:
breast = pd.read_csv('data/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 683 neurônios:
Parameters: p=683, eta=0.1, lambda=30.61918367346939, C=0.4498432668969444
********* Results ELM-IMA **************
Acc train: 0.9772+/-0.0028
Acc test: 0.9650+/-0.0253
Iterations: 234.7000+/-53.7048
Iterations IMA: 20.0000+/-0.0000
Updates: 3269.9000+/-1258.4215
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9841+/-0.0036
Acc test: 0.9621+/-0.0332
Iterations: 187.7000+/-13.5281
Iterations IMA: 20.0000+/-0.0000
Updates: 2228.4000+/-233.4657
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.8638+/-0.0285
Margin: 0.005836700+/-0.000603352
********* Results SVM **************
Acc train: 0.9733+/-0.0030
Acc test: 0.9723+/-0.0220
Margin: 0.000000+/-0.000000



Experimento com 341 neurônios:
Parameters: p=341, eta=0.1, lambda=42.86285714285714, C=0.4498432668969444
********* Results ELM-IMA **************
Acc train: 0.9756+/-0.0037
Acc test: 0.9635+/-0.0278

In [45]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1
run(X, y)

Experimento com 214 neurônios:
Parameters: p=214, eta=0.1, lambda=18.375510204081632, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.7617+/-0.0487
Acc test: 0.7576+/-0.1059
Iterations: 230.2000+/-23.1465
Iterations IMA: 20.0000+/-0.0000
Updates: 4348.8000+/-327.2549
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9257+/-0.0469
Acc test: 0.6446+/-0.0917
Iterations: 148.3000+/-24.0085
Iterations IMA: 20.0000+/-0.0000
Updates: 2007.5000+/-447.9628
Margin: 0.000000056+/-0.000000168
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6255+/-0.0973
Margin: 0.000006324+/-0.000002761
********* Results SVM **************
Acc train: 0.8344+/-0.0158
Acc test: 0.7996+/-0.1092
Margin: 0.000000+/-0.000000



Experimento com 107 neurônios:
Parameters: p=107, eta=0.1, lambda=61.228367346938775, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.7440+/-0.0536
Acc test: 0.6868+/-0.096

In [27]:
spam = pd.read_csv('data/spam/spambase.data', header=None, sep=",", engine='python')
y = spam[57].to_numpy()
X = spam.drop([57], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=53.06591836734693, C=5.963623316594643
********* Results ELM-IMA **************
Acc train: 0.9347+/-0.0093
Acc test: 0.9250+/-0.0252
Iterations: 332.3000+/-23.9084
Iterations IMA: 20.0000+/-0.0000
Updates: 44468.1000+/-2986.6246
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9295+/-0.0119
Acc test: 0.9144+/-0.0222
Iterations: 349.6000+/-22.2450
Iterations IMA: 20.0000+/-0.0000
Updates: 42602.0000+/-2407.6548
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.9607+/-0.0030
Acc test: 0.8955+/-0.0189
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.9611+/-0.0021
Acc test: 0.9315+/-0.0217
Margin: 0.000000+/-0.000000



Experimento com 500 neurônios:
Parameters: p=500, eta=0.1, lambda=81.63448979591837, C=5.963623316594643
********* Results ELM-IMA **************
Acc train: 0.9351+/-0.0066
Acc test: 0.9233+/-0.0